In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
model = "1호모델드랍down"
#model = "1호모델드랍down"

# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Q"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명
model_name = "{}_{}_k{}".format(model, freq, num_k)
model_name

'1호모델드랍down_Q_k3'

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트.csv", index_col=0)

In [4]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [5]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [6]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [7]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [8]:
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

list_data["2012-01"].columns

Index(['BW금액', 'CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '유상증자금액', '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액',
       '최대주주변경카운트', '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [9]:
drop_col_top = ["매출총이익","총자본","영업이익","당기순이익"]
drop_col_down =["최대주주변경을수반하는주식담보제공금액","경영권분쟁소송","업종변경카운트","특수관계인에대한자금대여금액"]

drop_together2 = tools.dict_data_drop(list_data.copy(), drop_col= drop_col_down, dropna=False)

drop_together2["2012-01"].columns

Index(['BW금액', 'CB금액', '불성실공시법인지정카운트', '상호변경카운트', '유상증자금액', '조회공시요구카운트',
       '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트', '타법인주식및출자증권취득결정금액', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [10]:
drop_togehter = tools.dict_data_drop(drop_together2.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)
scaled_together["2012-01"].columns

Index(['BW금액', 'CB금액', '불성실공시법인지정카운트', '상호변경카운트', '유상증자금액', '조회공시요구카운트',
       '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트', '타법인주식및출자증권취득결정금액', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [11]:
pd.set_option('display.max_columns', 130)

In [12]:
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=100, raw_data=scaled_together.copy())
model_save_dict

100%|██████████| 100/100 [00:07<00:00, 12.99it/s]


{'2012-01': {'num_of_cluseer': 3,
  'random_state': 27,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=27)},
 '2012-04': {'num_of_cluseer': 3,
  'random_state': 87,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=87)},
 '2012-07': {'num_of_cluseer': 3,
  'random_state': 76,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=76)},
 '2012-10': {'num_of_cluseer': 3,
  'random_state': 40,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=40)},
 '2013-01': {'num_of_cluseer': 3,
  'random_state': 90,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=90)},
 '2013-04': {'num_of_cluseer': 3,
  'random_state': 8,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=8)},
 '2013-07': {'num_of_cluseer': 3,
  'random_state': 10,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=10)},
 '2013-10': {'num_of_cluseer': 3,
  'random_state': 75,
  'model': KMeans(max_iter=50

## 모델 기록

In [13]:
size =evaluate.eval_cluster_size(model_save_dict)
size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

772.0
479.0


,2012-01,2012-04,2012-07,2012-10,2013-01,2013-04,2013-07,2013-10,2014-01,2014-04,2014-07,2014-10,2015-01,2015-04,2015-07,2015-10,2016-01,2016-04,2016-07,2016-10,2017-01,2017-04,2017-07,2017-10,2018-01,2018-04,2018-07,2018-10,2019-01,2019-04,2019-07,2019-10,2020-01,2020-04,2020-07,2020-10,2021-01,2021-04,2021-07,2021-10,2022-01,2022-04,2022-07
0,515.0,486.0,479.0,573.0,497.0,518.0,515.0,597.0,536.0,580.0,531.0,501.0,625.0,523.0,627.0,613.0,531.0,629.0,642.0,682.0,665.0,569.0,675.0,669.0,589.0,620.0,628.0,541.0,578.0,580.0,597.0,686.0,730.0,766.0,701.0,752.0,761.0,706.0,733.0,563.0,726.0,740.0,564.0
1,572.0,495.0,556.0,528.0,491.0,482.0,580.0,499.0,499.0,499.0,596.0,541.0,519.0,589.0,551.0,542.0,561.0,533.0,544.0,535.0,531.0,563.0,586.0,582.0,573.0,549.0,563.0,646.0,733.0,733.0,594.0,537.0,702.0,504.0,737.0,693.0,694.0,582.0,663.0,772.0,575.0,578.0,764.0
2,507.0,623.0,572.0,507.0,617.0,605.0,505.0,520.0,583.0,539.0,495.0,610.0,511.0,554.0,508.0,562.0,634.0,580.0,571.0,562.0,589.0,674.0,572.0,599.0,700.0,697.0,696.0,732.0,605.0,606.0,751.0,745.0,539.0,706.0,561.0,569.0,570.0,755.0,660.0,720.0,755.0,729.0,711.0
기업수,1594.0,1604.0,1607.0,1608.0,1605.0,1605.0,1600.0,1616.0,1618.0,1618.0,1622.0,1652.0,1655.0,1666.0,1686.0,1717.0,1726.0,1742.0,1757.0,1779.0,1785.0,1806.0,1833.0,1850.0,1862.0,1866.0,1887.0,1919.0,1916.0,1919.0,1942.0,1968.0,1971.0,1976.0,1999.0,2014.0,2025.0,2043.0,2056.0,2055.0,2056.0,2047.0,2039.0


In [14]:
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_man, freq=freq)
eval_one.to_csv("{}_관리종목.csv".format(model_name))
eval_one_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

In [15]:
eval_two,eval_two_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_war, freq=freq)
eval_two.to_csv("{}_투자환기종목.csv".format(model_name))
eval_two_original.to_csv("{}_투자환기종목전체대비.csv".format(model_name))

In [16]:
# 횡령인 경우
eval_three, eval_three_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_emb, freq=freq)
eval_three.to_csv("{}_횡령배임혐의.csv".format(model_name))
eval_three_original.to_csv("{}_횡령배임혐의전체대비.csv".format(model_name))

In [17]:
print(evaluate.calculate_catch_probability(eval_one, eval_two, None))#, eval_three)
print(evaluate.calculate_match_probability(eval_one, eval_two,None))# eval_three)
evaluate.calculate_catch_probability(eval_one_original, eval_two_original, None)#eval_three_original)

0.8870515181385341
1.0


0.11148432369514868

In [18]:
def pct(df):
    return df / np.sum(df)

dist = evaluate.eval_cluster_distance(eval_one, scaled_together, model_save_dict)
dist = dist.apply(pct, axis=1)
dist2= dist.mean(axis=0).sort_values(ascending=False)

dist.to_csv("{}_거리분해.csv".format(model_name))
dist2.to_csv("{}_거리분해평균.csv".format(model_name))

dist2

매출총이익                 0.163774
총자본                   0.141418
영업이익                  0.129202
매출채권                  0.106662
판매 및 관리비              0.104560
당기순이익                 0.103490
영업현금흐름                0.083829
최대주주비율                0.070496
결산기간_주가변동률            0.020892
차입금의존도                0.018770
CB금액                  0.017411
최대주주비율_변동성            0.009649
조회공시요구카운트             0.008885
결산기간_거래량증가            0.006834
최대주주변경카운트             0.005368
BW금액                  0.002103
불성실공시법인지정카운트          0.002084
타법인주식및출자증권취득결정금액      0.001503
유상증자금액                0.001291
최대주주변경을수반하는주식양수도금액    0.001011
상호변경카운트               0.000767
dtype: float64

예측하는 거 만들기

사용할 기간을 주면 -> 이후 가능한 끝까지의 값을 예측하고, 결과를 리턴하기

In [19]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2012-04예측,2012-07예측,2012-10예측,2013-01예측,2013-04예측,2013-07예측,2013-10예측,2014-01예측,2014-04예측,2014-07예측,2014-10예측,2015-01예측,2015-04예측,2015-07예측,2015-10예측,2016-01예측,2016-04예측,2016-07예측,2016-10예측,2017-01예측,2017-04예측,2017-07예측,2017-10예측,2018-01예측,2018-04예측,2018-07예측,2018-10예측,2019-01예측,2019-04예측,2019-07예측,2019-10예측,2020-01예측,2020-04예측,2020-07예측,2020-10예측,2021-01예측,2021-04예측,2021-07예측,2021-10예측
0,0.821429,0.1,0.861111,0.000000,0.090909,0.815789,0.857143,0.046512,0.851064,0.139535,0.860465,0.023810,0.104167,0.043478,0.046512,0.069767,0.051282,0.122449,0.211538,0.153846,0.086420,0.047059,0.121951,0.116279,0.867347,0.938776,0.956522,0.94382,0.848485,0.838384,0.858824,0.885057,0.047059,0.948718,0.042254,0.876923,0.871795,0.100000,0.75
1,0.089286,0.8,0.083333,0.914286,0.818182,0.026316,0.095238,0.883721,0.021277,0.023256,0.116279,0.904762,0.875000,0.065217,0.930233,0.906977,0.897436,0.061224,0.038462,0.038462,0.061728,0.835294,0.829268,0.046512,0.010204,0.000000,0.010870,0.00000,0.121212,0.151515,0.011765,0.011494,0.941176,0.025641,0.915493,0.092308,0.076923,0.033333,0.00
2,0.089286,0.1,0.055556,0.085714,0.090909,0.157895,0.047619,0.069767,0.127660,0.837209,0.023256,0.071429,0.020833,0.891304,0.023256,0.023256,0.051282,0.816327,0.750000,0.807692,0.851852,0.117647,0.048780,0.837209,0.122449,0.061224,0.032609,0.05618,0.030303,0.010101,0.129412,0.103448,0.011765,0.025641,0.042254,0.030769,0.051282,0.866667,0.25


고위험 클러스터의 평균 재무, 공시 상황 출력

* eval_cluster Result의 결과를 받는다 / model_save_dict / raw_data -> drop_together (스케일 전 데이터를 받아야함)

In [20]:
dict = evaluate.see_cluster_info(model_save_dict, eval_one, drop_togehter)

data = pd.DataFrame()
col = dict.keys()

for t, df in dict.items():
    data = pd.concat([data, df], axis=1)
data.columns = col
data.T.to_csv("{}_평균정보.csv".format(model_name))
data.T.mean().to_csv("{}_평균정보의평균.csv".format(model_name))
data.T

,BW금액,CB금액,불성실공시법인지정카운트,상호변경카운트,유상증자금액,조회공시요구카운트,최대주주변경을수반하는주식양수도금액,최대주주변경카운트,타법인주식및출자증권취득결정금액,결산기간_거래량증가,결산기간_주가변동률,당기순이익,매출채권,매출총이익,영업이익,영업현금흐름,차입금의존도,총자본,최대주주비율,판매 및 관리비,최대주주비율_변동성
2012-01,2.357227e+09,4.432010e+08,0.126214,0.042718,2.251015e+09,0.549515,7.347218e+08,0.153398,2.040092e+09,inf,1.037900,-3.673085e+06,1.967684e+07,2.859854e+05,-2.253650e+06,8.215927e+01,27.707044,4.570139e+07,30.764379,2.539639e+06,-0.000375
2012-04,2.498526e+09,4.451758e+08,0.119192,0.050505,5.137350e+09,0.561616,7.060246e+08,0.175758,3.229484e+09,inf,1.401032,-6.331077e+06,3.000729e+07,-2.922134e+05,-3.308436e+06,-4.598230e+03,28.194222,4.894319e+07,30.060068,3.016223e+06,0.000414
2012-07,3.440211e+09,3.852046e+08,0.144050,0.066806,4.106494e+09,0.555324,7.487205e+08,0.210856,3.570137e+09,inf,1.663449,-8.160467e+06,3.514942e+07,-3.729117e+05,-4.019658e+06,1.269067e+05,31.452260,5.185700e+07,28.418412,3.647086e+06,0.004252
2012-10,3.001561e+09,3.295580e+08,0.121212,0.070076,2.827871e+09,0.465909,8.054093e+08,0.178030,3.522683e+09,inf,1.563543,-7.061036e+06,3.758566e+07,-2.379723e+05,-3.838969e+06,-6.251873e+05,29.488622,4.996677e+07,30.728720,3.601305e+06,0.005726
2013-01,2.464895e+09,6.086241e+08,0.118126,0.079430,1.524032e+09,0.472505,7.563763e+08,0.191446,2.402952e+09,inf,1.553499,-6.496226e+06,3.282975e+07,-6.986490e+05,-3.925012e+06,-1.515812e+06,29.727683,4.345122e+07,30.398437,3.224562e+06,0.005378
2013-04,1.864176e+09,7.974541e+08,0.115830,0.075290,4.672499e+09,0.378378,7.391525e+08,0.200772,1.427994e+09,inf,1.583958,-5.713482e+06,1.826280e+07,-2.651678e+05,-2.818324e+06,-8.650860e+05,28.672534,3.096614e+07,31.660275,2.549428e+06,0.007045
2013-07,9.934583e+08,1.350967e+09,0.093204,0.073786,5.236320e+09,0.388350,6.760514e+08,0.186408,1.485093e+09,inf,1.558848,-6.575589e+06,4.077813e+07,-1.917381e+05,-3.794323e+06,-2.266056e+06,30.550131,4.292986e+07,32.038828,3.598520e+06,0.008824
2013-10,1.783046e+08,1.541113e+09,0.104208,0.068136,6.097982e+09,0.380762,7.023495e+08,0.210421,1.352604e+09,inf,1.641452,-6.963075e+06,4.432155e+07,-2.484626e+05,-3.934141e+06,-2.259142e+06,31.327009,4.704411e+07,30.811888,3.681483e+06,inf
2014-01,1.704664e+08,1.463849e+09,0.089552,0.061567,4.348308e+09,0.343284,4.962880e+08,0.167910,1.309566e+09,inf,1.314217,-5.682374e+06,1.734802e+07,-2.827035e+05,-2.634066e+06,-1.090800e+06,28.368218,3.002464e+07,31.893201,2.351362e+06,inf
2014-04,1.884601e+08,1.598660e+09,0.074212,0.064935,1.785950e+09,0.419295,5.804615e+08,0.159555,1.519943e+09,inf,1.211584,-2.284902e+06,1.724027e+07,1.470343e+06,-1.038320e+06,-3.642804e+05,28.907379,3.994924e+07,32.397039,2.508662e+06,inf


In [21]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_man["2021-01"], fig_title="{}_관리종목".format(model_name))

In [22]:
MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_war["2021-01"], fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000020,0.781382,-0.053990,0.259324,2,2.5
A000040,-0.284519,0.337303,-0.391036,0,2.5
A000050,0.768144,-0.029194,0.113006,2,2.5
A000070,1.217317,0.279813,0.058769,2,2.5
A000080,1.151936,0.202852,0.157660,2,2.5
...,...,...,...,...,...
A353810,0.054002,0.027135,0.024163,1,2.5
A354200,-0.958994,-0.115552,0.235182,0,2.5
A357780,0.919728,-0.353254,-0.439308,2,2.5
A363280,0.552340,-0.133453,-0.282295,2,2.5
